In [2]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import AutoTokenizer
import spacy

In [3]:
spacy.prefer_gpu()
nlp = spacy.load('../model')

In [5]:
df = pd.read_pickle('../data/label_data.pk1')

In [21]:
df.head()

,comment,user,date_time,sub_title,clean_comments,comment_len,entities,polarity,subjectivity,title,...,entities_both,len_entities,label_h,score_h,date,year_month,label_t,output_v,score_v,label_v
0,Seems like they misunderstood what is going on...,Worms_sg,1.647150e+09,NTU used as example of university gaming ranki...,Seems like they misunderstood what is going on...,702,[The Straits Times],0.094773,0.293409,NTU used as example of university gaming ranki...,...,[The Straits Times],1,0,0.628446,2022-03-13 05:43:22,2022-03,1,"{'neg': 0.059, 'neu': 0.84, 'pos': 0.101, 'com...",0.5423,1
1,"On one hand, such research isn't necessarily a...",rcRollerCoaster,1.647162e+09,NTU used as example of university gaming ranki...,"On one hand, such research isn't necessarily a...",2162,[Ministry of Education],0.063453,0.433367,NTU used as example of university gaming ranki...,...,[Ministry of Education],1,0,0.480066,2022-03-13 08:57:22,2022-03,1,"{'neg': 0.055, 'neu': 0.793, 'pos': 0.152, 'co...",0.9789,1
2,"PAP: And this is why, boys and girls, we saved...",nellynai,1.647143e+09,"Study: Iceland, Norway, and Singapore had no e...","PAP: And this is why, boys and girls, we saved...",288,"[Trace Together, People's Action Party]",0.283333,0.566667,"Study: Iceland, Norway, and Singapore had no e...",...,"[Trace Together, People's Action Party]",2,1,0.772632,2022-03-13 03:50:14,2022-03,1,"{'neg': 0.043, 'neu': 0.739, 'pos': 0.218, 'co...",0.9098,1
3,"The context is quite different leh. Norway, li...",Pyrrylanion,1.647149e+09,"Study: Iceland, Norway, and Singapore had no e...","The context is quite different leh. Norway, li...",3196,[Trace Together],0.139835,0.524475,"Study: Iceland, Norway, and Singapore had no e...",...,[Trace Together],1,0,0.521701,2022-03-13 05:20:02,2022-03,1,"{'neg': 0.063, 'neu': 0.764, 'pos': 0.173, 'co...",0.9968,1
4,Also the early screening work done by the EDB....,Worms_sg,1.647159e+09,"Study: Iceland, Norway, and Singapore had no e...",Also the early screening work done by the EDB....,1525,"[Government, The Straits Times, Ministry of He...",0.083787,0.357625,"Study: Iceland, Norway, and Singapore had no e...",...,"[Government, The Straits Times, Ministry of He...",3,0,0.785580,2022-03-13 08:11:39,2022-03,1,"{'neg': 0.017, 'neu': 0.907, 'pos': 0.076, 'co...",0.9100,1


In [24]:
df2 = df[df.entities_both.map(lambda x: len(set(x)))==1]

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149254 entries, 0 to 174462
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   comment         149254 non-null  object        
 1   user            149254 non-null  object        
 2   date_time       149254 non-null  float64       
 3   sub_title       149254 non-null  object        
 4   clean_comments  149254 non-null  object        
 5   comment_len     149254 non-null  int64         
 6   entities        149254 non-null  object        
 7   polarity        149254 non-null  float64       
 8   subjectivity    149254 non-null  float64       
 9   title           149254 non-null  object        
 10  title_ents      149254 non-null  object        
 11  len_ents        149254 non-null  int64         
 12  entities_both   149254 non-null  object        
 13  len_entities    149254 non-null  int64         
 14  label_h         149254 non-null  int

In [27]:
df2.head(50)

,comment,user,date_time,sub_title,clean_comments,comment_len,entities,polarity,subjectivity,title,...,entities_both,len_entities,label_h,score_h,date,year_month,label_t,output_v,score_v,label_v
0,Seems like they misunderstood what is going on...,Worms_sg,1.647150e+09,NTU used as example of university gaming ranki...,Seems like they misunderstood what is going on...,702,[The Straits Times],0.094773,0.293409,NTU used as example of university gaming ranki...,...,[The Straits Times],1,0,0.628446,2022-03-13 05:43:22,2022-03,1,"{'neg': 0.059, 'neu': 0.84, 'pos': 0.101, 'com...",0.5423,1
1,"On one hand, such research isn't necessarily a...",rcRollerCoaster,1.647162e+09,NTU used as example of university gaming ranki...,"On one hand, such research isn't necessarily a...",2162,[Ministry of Education],0.063453,0.433367,NTU used as example of university gaming ranki...,...,[Ministry of Education],1,0,0.480066,2022-03-13 08:57:22,2022-03,1,"{'neg': 0.055, 'neu': 0.793, 'pos': 0.152, 'co...",0.9789,1
3,"The context is quite different leh. Norway, li...",Pyrrylanion,1.647149e+09,"Study: Iceland, Norway, and Singapore had no e...","The context is quite different leh. Norway, li...",3196,[Trace Together],0.139835,0.524475,"Study: Iceland, Norway, and Singapore had no e...",...,[Trace Together],1,0,0.521701,2022-03-13 05:20:02,2022-03,1,"{'neg': 0.063, 'neu': 0.764, 'pos': 0.173, 'co...",0.9968,1
5,HK has more restrictions? Maybe lockdown style...,DuePomegranate,1.647159e+09,"Study: Iceland, Norway, and Singapore had no e...",HK has more restrictions? Maybe lockdown style...,421,[Trace Together],-0.042857,0.700000,"Study: Iceland, Norway, and Singapore had no e...",...,[Trace Together],1,-1,0.871669,2022-03-13 08:16:14,2022-03,0,"{'neg': 0.136, 'neu': 0.832, 'pos': 0.032, 'co...",-0.8462,0
6,> Commentary: Our fears are driving us to the...,sneakpeek_bot,1.647142e+09,Commentary: Our fears are driving us to the A&...,> Commentary: Our fears are driving us to the...,1760,[Ministry of Health],0.045982,0.412351,Commentary: Our fears are driving us to the A&...,...,[Ministry of Health],1,0,0.487883,2022-03-13 03:30:17,2022-03,0,"{'neg': 0.133, 'neu': 0.8, 'pos': 0.067, 'comp...",-0.9727,0
7,"> As inflation rises, how much more are you p...",sneakpeek_bot,1.647140e+09,"As inflation rises, how much more are you payi...","> As inflation rises, how much more are you p...",2320,[CNA],0.129060,0.488034,"As inflation rises, how much more are you payi...",...,[CNA],1,0,0.510323,2022-03-13 03:00:19,2022-03,1,"{'neg': 0.009, 'neu': 0.899, 'pos': 0.093, 'co...",0.9881,1
8,IMO it's the lack of practice of standing up f...,ddeng,1.647140e+09,War in Ukraine: Mixed public opinions reveal p...,IMO it's the lack of practice of standing up f...,600,[National Service],0.184659,0.530682,War in Ukraine: Mixed public opinions reveal p...,...,[National Service],1,-1,0.582095,2022-03-13 02:55:13,2022-03,1,"{'neg': 0.077, 'neu': 0.865, 'pos': 0.058, 'co...",0.2067,1
9,An argument for a ill-informed and politically...,junkredpuppy,1.647143e+09,War in Ukraine: Mixed public opinions reveal p...,An argument for a ill-informed and politically...,126,[People's Action Party],0.166667,0.222222,War in Ukraine: Mixed public opinions reveal p...,...,[People's Action Party],1,-1,0.617348,2022-03-13 03:37:50,2022-03,1,"{'neg': 0.201, 'neu': 0.684, 'pos': 0.115, 'co...",-0.2500,1
10,Propaganda from the PAP you mean.,YaleoverNUS,1.647148e+09,War in Ukraine: Mixed public opinions reveal p...,Propaganda from the PAP you mean.,33,[People's Action Party],-0.206250,0.393750,War in Ukraine: Mixed public opinions reveal p...,...,[People's Action Party],1,-1,0.636429,2022-03-13 05:01:33,2022-03,-1,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.2500,-1
11,"Sure, things you do not like is perpetuating t...",YaleoverNUS,1.647151e+09,War in Ukraine: Mixed public opinions reveal p...,"Sure, things you do not like is perpetuating t...",310,[National Service],0.106250,0.594444,War in Ukraine: Mix